In [ ]:
import pymc3 as pm
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import arviz as az
import statsmodels.formula.api as smf
%config InlineBackend.figure_format = 'retina'
az.style.use('arviz-darkgrid')

In [ ]:
d = pd.read_csv(r'D:\Bayes\resources\Rethinking\Data\WaffleDivorce.csv', sep=';')
# standardize predictor
d['MAMs'] = (d.MedianAgeMarriage - d.MedianAgeMarriage.mean()) / d.MedianAgeMarriage.std()
d.head()

In [ ]:
with pm.Model() as m51:
    alpha = pm.Normal('alpha', mu=10, sd=10)
    beta = pm.Normal('beta', mu=0, sd=1)
    sigma = pm.Uniform('sigma', lower=0, upper=10)
    mu = pm.Deterministic('mu', alpha + beta * d.MAMs)
    divorce = pm.Normal('divorce', mu=mu, sd=sigma, observed=d.Divorce)
    trace_51 = pm.sample(1000, tune=1000)

In [ ]:
az.plot_trace(trace_51, ['~mu'])

In [ ]:
mu_mean = trace_51['mu']

In [ ]:
plt.plot(d['MAMs'], d.Divorce, 'C0o')
plt.plot(d['MAMs'], mu_mean.mean(0), 'C1')
az.plot_hpd(d['MAMs'], mu_mean)
plt.xlabel('MedianAgeMarriage_s')
plt.ylabel('Divorce');

In [ ]:
az.summary(trace_51, ['~mu'])

In [ ]:
d['MRs'] = (d.Marriage - d.Marriage.mean()) / d.Marriage.std()

In [ ]:
with pm.Model() as m52:
    alpha = pm.Normal('alpha', mu=10, sd=10)
    beta = pm.Normal('beta', mu=0, sd=1)
    sigma = pm.Uniform('sigma', lower=0, upper=10)
    mu = pm.Deterministic('mu', alpha + beta * d.MRs)
    divorce = pm.Normal('divorce', mu=mu, sd=sigma, observed=d.Divorce)
    trace_52 = pm.sample(1000, tune=1000)

In [ ]:
az.plot_trace(trace_52, ['~mu'])

In [ ]:
mu_mean = trace_52['mu']

d.plot('MRs', 'Divorce', kind='scatter', xlim = (-2, 3))
plt.plot(d.MRs, mu_mean.mean(0), 'C1')
az.plot_hpd(d.MRs, mu_mean);

In [ ]:
az.summary(trace_52, ['~mu'])

In [ ]:
with pm.Model() as m53:
    alpha = pm.Normal('alpha', mu=10, sd=10)
    betaA = pm.Normal('betaA', mu=0, sd=1)
    betaR = pm.Normal('betaR', mu=0, sd=1)
    sigma = pm.Uniform('sigma', lower=0, upper=10)
    mu = pm.Deterministic('mu', alpha + betaA * d.MAMs + betaR * d.MRs)
    divorce = pm.Normal('divorce', mu=mu, sd=sigma, observed=d.Divorce)
    trace_53 = pm.sample(1000, tune=1000)

In [ ]:
az.plot_trace(trace_53, ['~mu'])

In [ ]:
az.summary(trace_53, ['~mu'])

In [ ]:
pm.forestplot(trace_53, var_names=['~mu'])

In [ ]:
with pm.Model() as m54:
    alpha = pm.Normal('alpha', mu=0, sd=10)
    betaA = pm.Normal('betaA', mu=0, sd=1)
    sigma = pm.Uniform('sigma', lower=0, upper=10)
    mu = pm.Deterministic('mu', alpha + betaA * d.MAMs)
    divorce = pm.Normal('divorce', mu=mu, sd=sigma, observed=d.MRs)
    trace_54 = pm.sample(1000, tune=1000)

In [ ]:
pm.traceplot(trace_54, ['~mu'])

In [ ]:
mu_pred = trace_54['mu'].mean(0)
residuals = d.MRs - mu_pred

In [ ]:
idx = np.argsort(d.MAMs)
d.plot('MAMs', 'MRs', kind='scatter', xlim = (-3, 3), ylim = (-3, 3))
plt.plot(d.MAMs[idx], mu_pred[idx], color='black')
plt.vlines(d.MAMs, mu_pred, mu_pred + residuals, colors='grey');

In [ ]:
with pm.Model() as model_532:
    a = pm.Normal('a', mu=10, sd=10)
    bA = pm.Normal('bA', mu=0, sd=1, shape=2)
    sigma = pm.Uniform('sigma', lower=0, upper=10)
    mu = pm.Deterministic('mu', a + bA[0] * d.MRs + bA[1] * d.MAMs)
    divorce = pm.Normal('Divorce', mu=mu, sd=sigma, observed=d.Divorce)
    trace_532 = pm.sample(1000, tune=1000)

In [ ]:
A_avg = np.linspace(-3, 3, 100)

mu_pred = trace_53['alpha'] + trace_53['betaR'] * A_avg[:,None]
divorce_ = stats.norm.rvs(mu_pred, trace_53['sigma'])

plt.plot(R_avg, mu_pred.mean(1), 'C0')
az.plot_hpd(R_avg, mu_pred.T, credible_interval=0.89)
az.plot_hpd(R_avg, divorce_.T, credible_interval=0.89)

plt.xlabel('MR')
plt.ylabel('Divorce')
plt.title('MAMs = 0');

In [ ]:
R_avg = np.linspace(-3, 3, 100)

mu_pred = trace_53['alpha'] + trace_53['betaA'] * R_avg[:,None]
divorce_ = stats.norm.rvs(mu_pred, trace_53['sigma'])

plt.plot(R_avg, mu_pred.mean(1), 'C0')
az.plot_hpd(R_avg, mu_pred.T, credible_interval=0.89)
az.plot_hpd(R_avg, divorce_.T, credible_interval=0.89)

plt.xlabel('MAMs')
plt.ylabel('Divorce')
plt.title('MRs = 0');

In [ ]:
mu_pred = trace_53['mu']

In [ ]:
divorce_pred = pm.sample_posterior_predictive(trace_53, samples=1000, model=m53)['divorce']
divorce_hpd = pm.hpd(divorce_pred)

In [ ]:
mu_hpd = az.hpd(mu_pred, credible_interval=0.95)
plt.errorbar(d.Divorce, divorce_pred.mean(0), yerr=np.abs(divorce_pred.mean(0)-mu_hpd.T), fmt='C0o')
plt.plot(d.Divorce, divorce_pred.mean(0), 'C0o')
plt.xlabel('Observed divorce')
plt.ylabel('Predicted divorce')
min_x, max_x = d.Divorce.min(), d.Divorce.max()
plt.plot([min_x, max_x], [min_x, max_x], 'k--')

In [ ]:
plt.figure(figsize=(10, 12))
residuals = d.Divorce - mu_pred.mean(0)
idx = np.argsort(residuals)
y_label = d.Loc[idx]
y_points = np.linspace(0, 1, 50)
plt.errorbar(residuals[idx], y_points, xerr=np.abs(divorce_pred.mean(0) - mu_hpd.T), fmt='C0o', lw=3)
plt.errorbar(residuals[idx], y_points, xerr=np.abs(divorce_pred.mean(0) - divorce_hpd.T), fmt='C0o', lw=2, alpha=0.5)
plt.yticks(y_points, y_label)
plt.vlines(0, 0, 1, 'grey')

In [ ]:
n = 100
x_real = stats.norm.rvs(size=n)
x_spur = stats.norm.rvs(x_real)
y = stats.norm.rvs(x_real)
d = pd.DataFrame([y, x_real, x_spur]).T
sns.pairplot(d)

In [ ]:
d = pd.read_csv(r'D:\Bayes\resources\Rethinking\Data\milk.csv', sep=';')

In [ ]:
d.head()

In [ ]:
dcc = d.dropna().copy()

In [ ]:
with pm.Model() as m55:
    alpha = pm.Normal('alpha', mu=0, sd=100)
    beta = pm.Normal('beta', mu=0, sd=1)
    sigma = pm.Uniform('sigma', lower=0, upper=1)
    mu = pm.Deterministic('mu', alpha + beta * dcc['neocortex.perc'])
    kcal = pm.Normal('kcal', mu=mu, sd=sigma, observed=dcc['kcal.per.g'])
    trace_55 = pm.sample(2000, tune=1000)

In [ ]:
pm.traceplot(trace_55, ['~mu'])

In [ ]:
az.summary(trace_55, ['~mu'], credible_interval=0.89).round(3)